# **EXAMEN FINAL DE SIS420**

## Nombre: Gonzales Suyo Franz Reinaldo
## C.U. 35-5335
## Carrera: Ing. Sistemas

# **APRENDIZAJE POR REFUERZO**

#

### Introducción al Ejemplo de Rompecabezas

El objetivo de este ejemplo es implementar un modelo de aprendizaje por refuerzo para resolver un rompecabezas de 4 filas por 5 columnas. Utilizaremos un enfoque basado en aprendizaje por refuerzo, similar al del juego de tres en raya, para enseñar a un agente a tomar decisiones óptimas que maximicen sus posibilidades de resolver el rompecabezas.


### Descripción de la Implementación

### Representación del Tablero del Rompecabezas:

Se creará una clase PuzzleBoard para representar el tablero de 4 filas por 5 columnas.
Esta clase manejará la lógica del juego, incluyendo la validación de movimientos, actualización del estado del tablero, verificación del estado del juego (si está ganado, perdido o empatado), y reinicio del tablero para nuevas partidas.
Agente de Aprendizaje por Refuerzo:

Se implementará una clase PuzzleAgent que representará al agente de aprendizaje por refuerzo.
El agente utilizará una tabla de valores para estimar las probabilidades de éxito desde diferentes estados del tablero.
El agente tomará decisiones basadas en una política de exploración-explotación, eligiendo a veces movimientos aleatorios (exploración) y otras veces movimientos que maximicen el valor esperado (explotación).
La función de valor del agente se actualizará utilizando una fórmula basada en la diferencia temporal después de cada movimiento.

#### Clase del Juego:

Se definirá una clase PuzzleGame para gestionar las partidas entre dos agentes.
Esta clase permitirá que los agentes jueguen múltiples partidas entre ellos, facilitando el entrenamiento de los agentes a través de la retroalimentación obtenida de los resultados de las partidas.
Entrenamiento del Agente:

Los agentes jugarán un gran número de partidas entre ellos para aprender las mejores estrategias.
Durante el entrenamiento, los agentes ajustarán continuamente sus funciones de valor basadas en las recompensas recibidas y los estados visitados durante las partidas.
Evaluación y Visualización de la Función de Valor:

Después del entrenamiento, se evaluarán y ordenarán los estados del tablero por su valor estimado.
Se utilizará una herramienta como Pandas para visualizar la función de valor de los agentes y analizar su desempeño.

In [1]:
import numpy as np
import pandas as pd

### Paso 1: Definir el Tablero del Rompecabezas
Primero, definimos la clase PuzzleBoard para representar el tablero del rompecabezas.

In [2]:

class PuzzleBoard:
    def __init__(self):
        # Inicializa el estado del tablero como una matriz 4x5 de ceros
        self.state = np.zeros((4, 5))
    
    def valid_moves(self):
        # Retorna una lista de todas las posiciones vacías en el tablero
        return [(i, j) for j in range(5) for i in range(4) if self.state[i, j] == 0]

    def update(self, symbol, row, col):
        # Actualiza el tablero con el símbolo del jugador en la posición (row, col)
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("Movimiento ilegal!")

    def is_game_over(self):
        # Verifica si el juego ha terminado
        # Gana el jugador 1 si hay una fila o columna con suma 4
        if (self.state.sum(axis=0) == 4).sum() >= 1 or (self.state.sum(axis=1) == 4).sum() >= 1:
            return 1
        # Gana el jugador 2 si hay una fila o columna con suma -4
        if (self.state.sum(axis=0) == -4).sum() >= 1 or (self.state.sum(axis=1) == -4).sum() >= 1:
            return -1
        # Empate si no hay movimientos válidos
        if len(self.valid_moves()) == 0:
            return 0
        # El juego sigue en curso
        return None

    def reset(self):
        # Reinicia el tablero a su estado inicial
        self.state = np.zeros((4, 5))

### Paso 2: Definir el Agente de Aprendizaje por Refuerzo
Ahora, definimos la clase PuzzleAgent que implementa el aprendizaje por refuerzo para el rompecabezas.

In [3]:
class PuzzleAgent:
    def __init__(self, symbol, alpha=0.5, epsilon=0.1):
        self.symbol = symbol        # Símbolo del agente (1 o -1)
        self.alpha = alpha          # Tasa de aprendizaje
        self.epsilon = epsilon      # Probabilidad de exploración
        self.value_function = {}    # Función de valor (tabla de valores de estados)
        self.positions = []         # Lista de posiciones visitadas en la partida actual

    def reset(self):
        # Reinicia la lista de posiciones visitadas
        self.positions = []

    def move(self, board, explore=True):
        # Decide la siguiente jugada del agente
        valid_moves = board.valid_moves()
        if explore and np.random.uniform(0, 1) < self.epsilon:
            # Exploración: elige un movimiento aleatorio
            return valid_moves[np.random.choice(len(valid_moves))]
        # Explotación: elige el movimiento con mayor valor estimado
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(4 * 5))
            value = self.value_function.get(next_state, 0.5)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col

    def update(self, board):
        # Almacena el estado del tablero después de cada movimiento
        self.positions.append(str(board.state.reshape(4 * 5)))

    def reward(self, reward):
        # Actualiza la función de valor al final de la partida
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0.5
                
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]

### Paso 3: Definir la Clase del Juego
Definimos la clase PuzzleGame para gestionar las partidas entre dos agentes.

In [4]:
class PuzzleGame:
    def __init__(self, agent1, agent2):
        self.board = PuzzleBoard()  # Crea el tablero del juego
        self.agent1 = agent1        # Primer agente
        self.agent2 = agent2        # Segundo agente

    def play(self):
        # Juega una partida completa entre los dos agentes
        self.board.reset()          # Reinicia el tablero
        self.agent1.reset()         # Reinicia el agente 1
        self.agent2.reset()         # Reinicia el agente 2
        turn = 1
        while True:
            if turn == 1:
                row, col = self.agent1.move(self.board)
                self.board.update(self.agent1.symbol, row, col)
                self.agent1.update(self.board)
                result = self.board.is_game_over()
                if result is not None:
                    self.agent1.reward(result)
                    self.agent2.reward(-result)
                    return result
                turn = 2
            else:
                row, col = self.agent2.move(self.board)
                self.board.update(self.agent2.symbol, row, col)
                self.agent2.update(self.board)
                result = self.board.is_game_over()
                if result is not None:
                    self.agent1.reward(-result)
                    self.agent2.reward(result)
                    return result
                turn = 1

    def selfplay(self, num_games):
        # Hace que los agentes jueguen múltiples partidas entre ellos
        results = []
        for _ in range(num_games):
            result = self.play()
            results.append(result)
        return results


### Paso 4: Entrenamiento de los Agentes
Finalmente, entrenamos los agentes jugando múltiples partidas entre ellos.

In [5]:
# Creación de los agentes
agent1 = PuzzleAgent(symbol=1, epsilon=0.5)
agent2 = PuzzleAgent(symbol=-1)

# Creación del juego
game = PuzzleGame(agent1, agent2)

# Entrenamiento de los agentes
results = game.selfplay(50000)

In [6]:
# Evaluación y visualización de la función de valor del agente 1
funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

print(tabla)


                                                   estado    valor
0       [ 0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  ...  0.96875
1       [ 1.  1. -1.  1. -1.  1.  0.  1.  1. -1.  1. -...  0.93750
2       [ 1.  0. -1. -1.  1.  1. -1.  1. -1.  1.  1.  ...  0.93750
3       [ 1.  0. -1. -1. -1.  1.  1.  1. -1.  1.  1. -...  0.93750
4       [ 1.  0. -1.  1. -1.  1. -1.  1. -1.  1.  1. -...  0.93750
...                                                   ...      ...
317049  [ 0.  1. -1.  1. -1.  1.  1.  1. -1. -1.  1.  ...  0.06250
317050  [ 0. -1.  1. -1. -1.  1.  1. -1.  1.  1.  1.  ...  0.06250
317051  [ 1.  1.  1.  1. -1.  0. -1.  1. -1.  1. -1.  ...  0.06250
317052  [ 0.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1.  ...  0.06250
317053  [ 1.  0. -1.  1. -1.  1.  1.  1.  1. -1.  1. -...  0.06250

[317054 rows x 2 columns]


In [22]:
import pickle

with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Conclusiones:

